In [1]:
import os, sys
from os.path import isfile, join, abspath, basename
from optparse import OptionParser
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import json

pathname = os.path.dirname(sys.argv[0])
full_pathname = os.path.abspath(pathname)
split_pathname = full_pathname.split(sep="mvesc")
base_pathname = os.path.join(split_pathname[0], "mvesc")
parentdir = os.path.join(base_pathname, "ETL")
sys.path.insert(0,parentdir)
from mvesc_utility_functions import *

In [76]:
schema = 'public'
public_tables = ['INV_06_16_CO_M', 'INV_06_16_FR_M', 'INV_06_16_MA_M', 'INV_06_16_RV_M', 'INV_06_16_RW_M', 
                 'INV_06_16_TV_M', 'INV_06_16_WM_M', 'INV_10_16_CEVSD_M', 'INV_10_16_EM_M']
new_column_names = ['student_lookup', 'status', 'grade', 'gender', 'hmrm', 
                    'membership_code', 'description', 'school_year', 'district']

table_df = {}
with postgres_pgconnection_generator() as conn:
    with conn.cursor() as cursor:
        for tab in public_tables:
            nrows = 1000000
            sql_read_table = """select * from {s}."{t}" limit {n};""".format(s=schema, t=tab, n=nrows)
            table_df[tab] = pd.read_sql_query(sql_read_table, conn)
        table_df['INV_06_16_CO_M']['District'] = 'Coshocton' # add missed `district` column in table for CO
        codes_df = pd.read_sql_query("select * from public.\"INV_MembershipCodes\";", conn)
        outcome = pd.read_sql_query("select * from model.outcome;", conn)
df = pd.DataFrame()
for t in table_df:
    new_col_dict = {table_df[t].columns[i]:new_column_names[i] for i in range(len(new_column_names))}
    table_df[t] = table_df[t].rename(columns=new_col_dict)
    df = df.append(table_df[t])
    #df.drop('hmrm', axis=1, inplace=True)
df = df.drop('hmrm', axis=1)
df.reset_index(drop=True, inplace=True)
codes_converter = {231101:231001, 231105:231005}
for i in range(df.shape[0]):
    if df.membership_code[i] in codes_converter:
        df.ix[i, 'membership_code'] = codes_converter[df.membership_code[i]]

In [87]:
# update and clean columns
df.grade.unique()
grade_converter = {'06':6, '05':5, '04':4, '03':3, '02':2, '01':1, 'KG':0:, 'PS':-1, '12', '11', '09',
       '10', '07', '08', '23', 'UG', 'GR', '13'}

array(['06', '05', '04', '03', '02', '01', 'KG', 'PS', '12', '11', '09',
       '10', '07', '08', '23', 'UG', 'GR', '13'], dtype=object)

In [58]:
codes_df.head()

,membership_code,code,membership_group
0,120010,Early College High School,Post-secondary Enrollment Options Program
1,120020,Seniors to\r\nSophomores,Post-secondary Enrollment Options Program
2,151490,INTERV DUR SUMMER,Academic Intervention
3,151500,Summer Reading Program,Academic Intervention
4,152330,Student received intervention provided during ...,Academic Intervention


In [59]:
outcome.head()

,student_lookup,not_on_time,is_dropout,definite,cohort_10th,cohort_9th,cohort_8th,cohort_7th,cohort_6th
0,62830.0,1,0,NaN,2006.0,2006,NaN,NaN,NaN
1,58382.0,0,0,0.0,2006.0,2007,NaN,NaN,NaN
2,62594.0,1,0,NaN,2006.0,2006,NaN,NaN,NaN
3,60526.0,1,0,NaN,2006.0,2006,NaN,NaN,NaN
4,59594.0,0,0,0.0,2006.0,2006,NaN,NaN,NaN


In [61]:
students_not_in_codes = df[df.membership_code==410003].student_lookup.unique()

In [37]:
unique_codes = df.membership_code.unique()
code_not_in_codes = list(filter(lambda x: x not in list(codes_df.membership_code), unique_codes))

In [49]:
rows_not_in_codes = df[pd.Series([x in code_not_in_codes for x in df.membership_code])]
rows_not_in_codes.groupby(by=['membership_code', 'description']).count()

,,student_lookup,status,grade,gender,school_year,district
membership_code,description,,,,,,
231101,Title I Reading\r\n(Summer School),174,174,174,174,174,174
231105,Title I Mathematics\r\n(Summer School),261,261,261,261,261,261
410003,FFA (formerly\r\nFuture Farmers of\r\nAmerica),2754,2754,2754,2754,2754,2754


In [71]:
231105 in codes_converter

True

In [66]:
check_drop = outcome.merge(rows_not_in_codes, on=['student_lookup'], how='inner')

In [68]:
check_drop.sum(axis=0)

student_lookup                                           6.31788e+07
not_on_time                                                      235
is_dropout                                                        56
definite                                                          56
cohort_10th                                              3.50441e+06
cohort_9th                                                   3554921
cohort_8th                                                2.8829e+06
cohort_7th                                               2.19931e+06
cohort_6th                                               1.63633e+06
status             AAAAAAOOQAAAAAAAAAAAAAIAAAAAAAAAAAAAAAAAAAACAA...
grade              1010111210101011101010101011121011101110111210...
gender             MMMMMMFFMMMMMMMMMMMMMMMMMMMMMMMFFMMMMMMMMFFMMM...
membership_code                                            705974135
description        FFA (formerly\r\nFuture Farmers of\r\nAmerica)...
school_year        11/19/200820071